Method 2
========

- Compute the mean of each seen class.

- Learn the multi-output regression model with class attribute vector being the input and the class mean vector being the output (this will use the seen class attributes and their mean vectors).

- Apply the learned regression model to compute the mean of each unseen class.

- Apply the model to predict labels on unseen class test inputs.

- Compute classification accuracies.

- Note: You need to try several values of the regularization hyperparameter \lambda


Brief description of Dataset -->

X_seen=np.load('X_seen.npy') 	(40 x N_i x D): 40 feature matrices. X_seen[i] is the N_i x D feature matrix of seen class i

Xtest=np.load('Xtest.npy')	(6180, 4096): feature matrix of the test data.

Ytest=np.load('Ytest.npy',)	(6180, 1): ground truth labels of the test data

class_attributes_seen=np.load('class_attributes_seen.npy')	(40, 85): 40x85 matrix with each row being the 85-dimensional class attribute vector of a seen class.

class_attributes_unseen=np.load('class_attributes_unseen.npy')	(10, 85): 10x85 matrix with each row being the 85-dimensional class attribute vector of an  unseen class.




In [8]:
import numpy as np

In [9]:
""" Loading provided datasets"""

X_seen=np.load('X_seen.npy',allow_pickle=True,encoding='bytes')
Xtest=np.load('Xtest.npy',allow_pickle=True,encoding='bytes')
Ytest=np.load('Ytest.npy',allow_pickle=True,encoding='bytes')
class_attributes_seen=np.load('class_attributes_seen.npy',allow_pickle=True,encoding='bytes')
class_attributes_unseen=np.load('class_attributes_unseen.npy',allow_pickle=True,encoding='bytes')

In [10]:
"""reg_parameter is a list of hyperparameter λ ; λ values to consider in this method are given in problem"""

reg_parameter = [0.01, 0.1, 1, 10, 20, 50, 100]

""" calculating mean of seen classes for training reg. model by taking mean as output and class attribute vector as input"""
mean_of_seen_cls = []
for cls in X_seen:
    summation = np.zeros(4096,)
    for each_img in cls:
        summation = summation+each_img
    mean_of_seen_cls.append(summation/len(cls))
    
""" M is the matrix of mean of seen classes i.e. output in this model so M is output matrix"""
"""(40,4096) shape output matrix for 40 classes"""

M = np.array(mean_of_seen_cls).reshape(40,4096)

""" Dataset provided for matrix A"""
A = class_attributes_seen   
# a,b = np.shape(A)
"""Identity matrix of 85*85"""
I = np.identity(85)     




In [11]:
""" defining a function for accuracy :
(accuracy is the percentage of test inputs with correctly predicted classes)"""

def accuracy(arr1,arr2):
    """ this function takes 2 lists as input and calculates accuracy by maching corresponding values."""
    count = 0
    for value in range(len(arr1)):
        if(arr1[value]==arr2[value]):
            count = count+1
    return count*100/len(arr1)



In [12]:
""" initializing an empty list for storing accuracy of reg. model corresponding to each hyperparameter lambda"""

accuracy_list = []

""" In the nested loop below I'm calculating accuracy of each diff. model corresponding to each value of λ given"""

"""At first model learns weight matrix by closed form formula (given in problem) 
  then model makes prediction based on this matrix
  the predicted labels along with actual labels area passed in accracy function and calc. accuracy is stored in accuracy_list 
"""

for lmbda in reg_parameter:
    W = []
    mean_of_unseen_cls = []
    Ypred = []
    """ learning weight matrix"""
    W = np.linalg.inv(np.transpose(A) @ A + lmbda * I) @ np.transpose(A) @ M
    for cls in range(10):
        pred_mean = np.transpose(W) @ class_attributes_unseen[cls]
        mean_of_unseen_cls.append(pred_mean)
        
    """ taking one img at a time for prediction"""    
    for sample in Xtest:
        count=0
        minimum = 10**10
        while(count<10):
            L2_norm = np.linalg.norm(sample-np.array(mean_of_unseen_cls[count]))
            if(L2_norm < minimum):
                minimum = L2_norm
                pred_class = count+1
            count = count+1
        Ypred.append(pred_class)
    accuracy_list.append(accuracy(Ypred,Ytest))
    
print(len(accuracy_list))

7


In [13]:
"""Accuracy for each λ"""

for i in range(len(accuracy_list)):
    print("Accracy for λ =",reg_parameter[i]," is: ",accuracy_list[i])

max_acc_index = accuracy_list.index(max(accuracy_list))
print("The maximum Accuracy is ",accuracy_list[max_acc_index]," obtained on λ =",reg_parameter[max_acc_index])

Accracy for λ = 0.01  is:  58.090614886731395
Accracy for λ = 0.1  is:  59.54692556634304
Accracy for λ = 1  is:  67.3948220064725
Accracy for λ = 10  is:  73.28478964401295
Accracy for λ = 20  is:  71.68284789644012
Accracy for λ = 50  is:  65.08090614886731
Accracy for λ = 100  is:  56.47249190938511
The maximum Accuracy is  73.28478964401295  obtained on λ = 10
